In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit #tine cont si de cls adica ia acelasi procent si de spam si de not spam
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [2]:
def get_data(filename):

    df = pd.read_csv(filename, delimiter=',') #citim fisierul csv folosind delimitatorul ,
    labels = df['type'].values #preluam labels .values sub forma unui np array
    mesages = df['text'].values #preluam mesajele

    pos_examples = np.sum(labels == 'ham') / labels.shape[0] # aduna toate exemplele pozitive 1 adica ham si le 
    #impartim la nr total de exemp ca sa obtinem procent, labels.shapep[0] tuplu cu prima pozitie nr de linii si a doua nr de coloane
    neg_examples = 1 - pos_examples #procent exemple negative
    print("procent exemple pozitive:",pos_examples)
    print("procent exemple negative:",neg_examples)
    

    shuffle_stratified = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
    #daca in train avem 86% nospam si 13% spam ar fi indicat ca si la test aceleasi procente
    #n_splits=1 ca doar un split vrem
    #test_size=procentul pe care vrem sa l pastram in validare
    #generaza 2 liste pe care le poti imparti in train si test
    for train_index, test_index in shuffle_stratified.split(mesages, labels):
        msg_train, msg_test = mesages[train_index], mesages[test_index]
        labels_train, labels_test = labels[train_index], labels[test_index]
    #splituim datasetul in train si test 
    
    y_train = np.int32(labels_train == 'ham')
    y_test = np.int32(labels_test == 'ham')
    #traducem din 'ham' in 1 (valori numerice)
    #practic facem un array cu 1 si 0, iar noi am pus
    #ca atunci cand labels_train='ham' sa avem "1" adica true
    #iar cand avem "spam" va da 0 adica false
    print("valorile de 1 sunt ham si 0 sunt spam:",y_train)

    return msg_train, y_train, msg_test, y_test


msg_train, y_train, msg_test, y_test = get_data('/kaggle/input/spam-ham-sms-dataset/sms_spam.csv')

count_vectorizer = CountVectorizer(lowercase=True, analyzer='word', stop_words='english')
#colectie de mesaje sms intr o matrice de numaratori de termeni
#functie folosita la prelucrarea limbajului natural
#lowercase true pt ca sa converteasca toate literele in lowercase
#analyzer=word ca sa numere cuvintele, analiza se face la nivel de cuvant
#stop_words="english" elimina cuvintele comune in engleza (and, the, is, etc.)
count_vectorizer.fit(msg_train) #il antrenam
X_train = count_vectorizer.transform(msg_train) 
#matricea de feature uri pt train
X_test = count_vectorizer.transform(msg_test)
#matricea de feature uri pt test

model = MultinomialNB(alpha=0.01)
#lucreaza bn cu feature uri de tip count, cum am numarat noi cuvintele
#am modificat alfa pt acea problema unde o probabilitate conditionata e 0
#pt ca nu au fost intalnite in train, corectie de tip Lidstone <1
model.fit(X_train, y_train)

predictions = model.predict(X_test)
#realizam predictii pe datele de test

print("acuratetea:",accuracy_score(y_test, predictions))
#printam acuratetea modelului
print(classification_report(y_test, predictions))
#calc automat recall precizie si f1score, cel mai imp fiind
#recall ul maxim pt not spam, 0 spam si 1 not spam
#precizie mare pt spam

#În cazul în care setul nostru de testare este balansat (50% exemple pozitive - 50% exemple negative), acuratețea este o metodă bună de evaluare.
#Problema apare atunci când testăm pe un set de date debalansat (ex: 90 negative 10 pozitive). În astfel de situații acuratețea ne poate duce în eroare.

#precizia= tp/tp+fp "cate chiar erau spam din cele clasificate ca spam?"
#Precizia ne ajută să ne dăm seama de corectitudinea prezicerilor noastre pentru o anumită clasă. 
#Practic, această metrică răspunde la întrebarea: “Care este procentul de rezultate prezise pe o anumită clasă care au fost prezise corect?”.

#recall= tp/tp+fn "cate chiar erau not spam din toate cele chiar sunt not spam?"
#se vrea maxim pt ca e de dorit sa fim siguri ca un mail nu e spam, mai bn afisam in mail cateva spam decat sa nu afisam un mail important pt ca a crezut ca e spam
#Recall-ul ne indică câte rezultate corecte au fost prezise de model pentru o anumită clasă. 
#Întrebarea la care răspunde este: “Care este procentul de rezultate clasificate corect pentru o clasă din totalul de exemple din acea clasă?”.

#F1score=Această metrică ține cont atât de precizie cât și de recall și reprezintă media armonică între cele 2.


procent exemple pozitive: 0.8656233135456017
procent exemple negative: 0.13437668645439826
valorile de 1 sunt ham si 0 sunt spam: [1 1 1 ... 1 1 1]
acuratetea: 0.9802158273381295
              precision    recall  f1-score   support

           0       0.92      0.93      0.93       149
           1       0.99      0.99      0.99       963

    accuracy                           0.98      1112
   macro avg       0.96      0.96      0.96      1112
weighted avg       0.98      0.98      0.98      1112

